# Spool up EMR Cluster

In [1]:
import boto3
import time
import sys
import botocore
import paramiko
import re
import os
import yaml, re

In [2]:
PATH = os.path.abspath(os.getcwd())
PATH

'/data/hail-on-AWS-spot-instances/src'

In [3]:
#Setup logging
import logging, bluebee
from bluebee import bgp

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
logFile = f'{PATH}/spoolup_EMR_cluster.log'
fh = logging.FileHandler(logFile)
fh.setLevel(logging.INFO)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)-18s-%(levelname)-8s %(message)s', datefmt='%d%b%Y %H:%M:%S')
ch.setFormatter(formatter)
fh.setFormatter(formatter)
# add the handlers to logger
logger.addHandler(ch)
logger.addHandler(fh)
# bgp.api.dump_curl = True
# bluebee.logger.setLevel(logging.DEBUG)

2021021016


In [6]:
#Get the configuration as a yaml object
c=yaml.load(open(PATH+"/config_EMR_spot.yaml"),Loader=yaml.SafeLoader)
logger.info(f'Configuration settings: {c}')

24Mar2021 21:26:58-INFO     Configuration settings: {'config': {'EMR_CLUSTER_NAME': 'tdeboer-hail', 'EC2_NAME_TAG': 'tdeboer-hail-EMR', 'OWNER_TAG': 'tdeboer-ilmn', 'PROJECT_TAG': 'GRE_on_ICA', 'REGION': 'us-east-1', 'MASTER_INSTANCE_TYPE': 'm4.large', 'WORKER_INSTANCE_TYPE': 'r4.4xlarge', 'WORKER_COUNT': '4', 'WORKER_BID_PRICE': '0.50', 'MASTER_HD_SIZE': '250', 'WORKER_HD_SIZE': '500', 'SUBNET_ID': '', 'S3_BUCKET': 's3n://ilmn-hail/', 'KEY_NAME': 'hail-ES-GRE', 'PATH_TO_KEY': '/data/', 'WORKER_SECURITY_GROUP': 'sg-0df1e5704ca2a8196', 'MASTER_SECURITY_GROUP': 'sg-0bab1202c0aa453b3', 'HAIL_VERSION': 'current'}}


In [7]:
# Spot instances and different CORE/MASTER instances
command='aws emr create-cluster --applications Name=Hadoop Name=Spark --tags \'project='+\
c['config']['PROJECT_TAG']+'\' \'Owner='+c['config']['OWNER_TAG']+\
'\' \'Name='+c['config']['EC2_NAME_TAG']+'\' --ec2-attributes \'{"KeyName":"'+\
c['config']['KEY_NAME']+'","InstanceProfile":"EMR_EC2_DefaultRole","SubnetId":"'+\
c['config']['SUBNET_ID']+'","EmrManagedSlaveSecurityGroup":"'+c['config']['WORKER_SECURITY_GROUP']+\
'","EmrManagedMasterSecurityGroup":"'+c['config']['MASTER_SECURITY_GROUP']+\
'"}\' --service-role EMR_DefaultRole --release-label emr-5.23.0 --log-uri \''+\
c['config']['S3_BUCKET']+'\' --name \''+c['config']['EMR_CLUSTER_NAME']+\
'\' --instance-groups \'[{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":'+\
c['config']['MASTER_HD_SIZE']+',"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"MASTER","InstanceType":"'+\
c['config']['MASTER_INSTANCE_TYPE']+'","Name":"Master-Instance"},{"InstanceCount":'+\
c['config']['WORKER_COUNT']+',"BidPrice":"'+c['config']['WORKER_BID_PRICE']+\
'","EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":'+\
c['config']['WORKER_HD_SIZE']+',"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"CORE","InstanceType":"'+\
c['config']['WORKER_INSTANCE_TYPE']+\
'","Name":"Core-Group"}]\' --configurations \'[{"Classification":"spark","Properties":{"maximizeResourceAllocation":"true"}},{"Classification":"yarn-site","Properties":{"yarn.nodemanager.vmem-check-enabled":"false"},"Configurations":[]}]\' --auto-scaling-role EMR_AutoScaling_DefaultRole --ebs-root-volume-size 32 --scale-down-behavior TERMINATE_AT_TASK_COMPLETION --region '+\
c['config']['REGION']+' --bootstrap-actions Path="s3://ilmn-hail/hail_bootstrapping/bootstrap_python36.sh"'
logger.info(f'Executing following command: \n{command}')

24Mar2021 21:27:21-INFO     Executing following command: 
aws emr create-cluster --applications Name=Hadoop Name=Spark --tags 'project=GRE_on_ICA' 'Owner=tdeboer-ilmn' 'Name=tdeboer-hail-EMR' --ec2-attributes '{"KeyName":"hail-ES-GRE","InstanceProfile":"EMR_EC2_DefaultRole","SubnetId":"","EmrManagedSlaveSecurityGroup":"sg-0df1e5704ca2a8196","EmrManagedMasterSecurityGroup":"sg-0bab1202c0aa453b3"}' --service-role EMR_DefaultRole --release-label emr-5.23.0 --log-uri 's3n://ilmn-hail/' --name 'tdeboer-hail' --instance-groups '[{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":250,"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"MASTER","InstanceType":"m4.large","Name":"Master-Instance"},{"InstanceCount":4,"BidPrice":"0.50","EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":500,"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"CORE","InstanceType":"r4.4xlarge","Name":"Core-Group"

In [9]:
cluster_id_json=os.popen(command).read()
#cluster_id=cluster_id_json.split(": \"",1)[1].split("\"\n")[0]
cluster_id=re.split('\s',cluster_id_json)[1]
# Gives EMR cluster information
client_EMR = boto3.client('emr', region_name=c['config']['REGION'])
logger.info(f'Created cluster with ID: "{cluster_id}"')

24Mar2021 21:27:40-INFO     Found credentials in shared credentials file: ~/.aws/credentials
24Mar2021 21:27:40-INFO     Created cluster with ID: "j-31IT4MS4HXYXB"


In [10]:
# Cluster state update
status_EMR='STARTING'
tic = time.time()
# Wait until the cluster is created
while (status_EMR!='EMPTY'):
	print('Creating EMR...')
	details_EMR=client_EMR.describe_cluster(ClusterId=cluster_id)
	status_EMR=details_EMR.get('Cluster').get('Status').get('State')
	print('Cluster status: '+status_EMR)
	time.sleep(30)
	if (status_EMR=='WAITING'):
		print('Cluster successfully created! Starting HAIL installation...')
		toc=time.time()-tic
		print("\n Total time to provision your cluster: %.2f "%(toc/60)+" minutes")
		break
	if (status_EMR=='TERMINATED_WITH_ERRORS'):
		sys.exit("Cluster un-successfully created. Ending installation...")

Creating EMR...
Cluster status: STARTING
Creating EMR...
Cluster status: STARTING
Creating EMR...
Cluster status: STARTING
Creating EMR...
Cluster status: STARTING
Creating EMR...
Cluster status: STARTING
Creating EMR...
Cluster status: STARTING
Creating EMR...
Cluster status: STARTING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: BOOTSTRAPPING
Creating EMR...
Cluster status: WAITING
Cluster successfully created! Starting HAIL installation...

 Total time to provision your cluster: 10.05  minutes


In [11]:
# Get public DNS from master node
master_dns=details_EMR.get('Cluster').get('MasterPublicDnsName')
master_IP=re.sub("-",".",master_dns.split(".")[0].split("ec2-")[1])
logger.info(f'Master IP address: {master_IP}')

24Mar2021 23:23:11-INFO     Master IP address: 54.172.151.112


In [17]:
# Copy the key into the master
command='scp -o \'StrictHostKeyChecking no\' -i '+\
c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem '+\
c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem hadoop@'+\
master_dns+':/home/hadoop/.ssh/id_rsa'
logger.info(f'Copying the keys to the instance with command: \n{command}')
assert os.system(command) == 0

24Mar2021 23:29:15-INFO     Copying the keys to the instance with command: 
scp -o 'StrictHostKeyChecking no' -i /data/hail-ES-GRE.pem ~/hail-ES-GRE.pem hadoop@ec2-54-172-151-112.compute-1.amazonaws.com:/home/hadoop/.ssh/id_rsa


In [16]:
# Copy the installation script into the master
command='scp -o \'StrictHostKeyChecking no\' -i '+\
c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem '+\
PATH+'/install_hail_and_python36.sh hadoop@'+master_dns+':/home/hadoop'
logger.info(f'Copying the hail installation script to the instance with command: \n{command}')
assert os.system(command) == 0

24Mar2021 23:26:47-INFO     Copying the hail installation script to the instance with command: 
scp -o 'StrictHostKeyChecking no' -i /data/hail-ES-GRE.pem /data/hail-on-AWS-spot-instances/src/install_hail_and_python36.sh hadoop@ec2-54-172-151-112.compute-1.amazonaws.com:/home/hadoop


In [36]:
#Install the software on master
logger.info(f'Installing hail et. al on Master')
logger.warning(f'This is the public JupyterLab link: "http://{master_IP}:8192"')
key = paramiko.RSAKey.from_private_key_file(c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem')
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=master_IP, username="hadoop", pkey=key)
#Run a quick test on the connection
stdin, stdout, stderr = client.exec_command('echo "Hello"')
assert stdout.readline() == 'Hello\n'

24Mar2021 23:54:07-INFO     Installing hail et. al on Master
24Mar2021 23:54:07-WARNING  This is the public JupyterLab link: "http://54.172.151.112:8192"
24Mar2021 23:54:07-INFO     Connected (version 2.0, client OpenSSH_7.4)
24Mar2021 23:54:07-INFO     Authentication (publickey) successful!


In [21]:
#Connect to the instance via ssh and execute the command
VERSION=c['config']['HAIL_VERSION']
command='./install_hail_and_python36.sh -v '+ VERSION
logger.info(f'Executing remote command: "{command}"')
stdin, stdout, stderr = client.exec_command('cd /home/hadoop/')
stdin, stdout, stderr = client.exec_command(command)

In [42]:
# close the client connection
logger.info('Closing the client connection')
client.close()

24Mar2021 23:56:27-INFO     Closing the client connection


In [43]:
logger.warning(f'Successfully started ENR cluster "{cluster_id}"')

24Mar2021 23:57:19-WARNING  Successfully started ENR cluster "j-31IT4MS4HXYXB"
